In [161]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import pickle
from scipy import stats

# Festlegen globaler Parameter der Simulation

In [162]:
# Breite der Zeitschritte
T_STEP = 15 
# Anzahl zu simulierender Fahrzeuge
EV_GES = 1000
# Anzahl an Tagen die simuliert werden sollen 
D_TOTAL = 1
# Tagtyp: 1 = Werktag, 2 = Samstag, 3 = Sonntag
TYPE_D = 1 
# Ladeszenario: 1 = nur Zuhause, 2 = Zuhause und Arbeit, 3 = Überall
CHARGE_SCEN = 1 
# Ladeleistung 
PCHARGE_SLOW = 3.7 

# durchschnittliche Fahrgeschwindigkeit
# Dummy Wert aus anderem Modell
EV_SPEED = 19

# Laden der Simulationsdaten

In [163]:
import pickle
import os
root = os.getcwd()+"\\Simulationsdaten"

# Verteilungsfunktion für initiale Abreise
initial_departure_model = pickle.load(open(root+"\\Modell_Initiale_Abfahrtszeit_Werktag.pickle", "rb"))

"""
Übergangswahrscheinlichkeiten:
- eine Liste pro Ausgangszustand mit einem Eintrag pro Zeitschritt -> 96 Einträge 
- jeder der 96 Einträge enthält 4 Werte die der jeweiligen relativen Übergangswahrscheinlichkeit entsprechen
    -> sortiert nach numerischer Repräsentation der Zustände
- Beispiel: transition_prob_home[10] = [0.2, 0.4, 0.1, 0.3] = [p1_2, p1_3, p1_4, p1_5] im Zeitintervall 9-10(dummy Werte)
"""
transition_prob_home = pickle.load(open(root+"\\Übergangswahrscheinlichkeiten_Zuhause.pickle", "rb"))

"""
Parameter Verteilungsfunktion Wegstrecken: 
- jeweils m*m Liste wobei Parameter der Verteilungsfunktion der Weglänge von i nach j in Reihe i-1 und Spalte j-1 zu finden sind
- Beispiel: loc[0][1] = Mittelwert der Weglängenverteilung Zustand Zuhause(1) nach Zustand Arbeit(2)
"""
# shape = Formparameter der Verteilung
dist_wd_shape_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Shape.pickle", "rb"))
# scale = Standardabweichung der Verteilung
dist_wd_scale_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Scale.pickle", "rb"))
# loc = Mittelwert der Verteilung
dist_wd_loc_ij = pickle.load(open(root+"\\Verteilung_Wegstrecken_Werktage_Loc.pickle", "rb"))

"""
Verteilungsfunktion der Aufenthaltsdauern in den einzelnen Zuständen:
- Liste mit dem jeweiligen Density Estimation Modell des Zustands in sortierter Reihenfolge
- Modell des Zustands i in Index i-1 
- Beispiel: Modell Aufenthaltsdauer Zustand 2(= Arbeit) in stay_duration_model[1]
"""
stay_duration_model = pickle.load(open(root+"\\Modelle_Aufenthaltsdauer_Werktag.pickle", "rb"))



# Initialisieren Fahrzeug

In [164]:
class Electric_Vehicle(object):
    model_df = pd.read_excel(r"C:\Users\thoma\Desktop\ev-modelling-repo\Data\EV_Modelle_Tabelle.xlsx", index_col='Modell')
    segments = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -9])
    # prob_segment einlesen und nicht bei jeder EV Initialisierung neu definieren 
    
    def __init__(self):
        segment = self.choose_segment()
        model = self.choose_model(segment)
        self.MODEL = model
        
        # Bug da Modelle mehrfach in Liste keine eindeutige Zuordnung sondern Array als output -> fixed -> elegantere Lösung?
        # Segment des Fahrzeugs
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Segment"]):
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"]
        else:
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"][0]
        
        # Batteriekapazität in kWh
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Batterie"]):
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"]
        else:
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"][0]
        
        # Verbrauch in kWh/100km
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Verbrauch"]):
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"]
        else:
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"][0]
        
        # Leistung (notwendig?) 
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Leistung"]):
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"]
        else:
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"][0]
            
        self.SOC = 100
        self.trip_no = 0
        self.trips = []
    
    def choose_segment(self): 
        # WICHTIG: Später errechnen und als globale Variable in Klasse speichern und nur Zugriff -> Performance
        
        # Beispieldaten der relativen Verteilung der Fharzeugsegmente aus MOP Studie 2016 
        prob_segment = {
        1 : 5.92,
        2 : 19.58,
        3 : 26.21,
        4 : 15.97,
        5 : 2.9,
        6 : 0.58,
        7 : 3.54,
        8 : 1.22,
        9 : 6.37,
        10 : 5.86,
        11 : 3.86,
        12 : 0.58,
        13 : 5.28,
        -9 : 2.13
        }
        
        # Rundungsfehler beseitigen -> sicherstellen dass sich W'keiten zu 1 aufsummieren
        p_ges = sum(val for key, val in prob_segment.items())
        p_rest = 100-p_ges
        prob_segment[-9] = prob_segment[-9]+p_rest
        
        # wähle p zufällig auf Basis gegebener W'keiten
        choice = np.random.choice(Electric_Vehicle.segments, p=[prob_segment[x]/100 for x in Electric_Vehicle.segments])
        
        # Vorerst Wahl des häufigsten Semgents bei keiner Angabe -> später überarbeiten
        if choice == -9: 
            choice = 3
        
        return choice
    
    def choose_model(self, segment):
        models = Electric_Vehicle.model_df
        
        # filtern der infragekommenden Fahrzeuge über Segment
        filt = models["Segment"] == segment
        choices = models[filt]
        
        # Wahl einse zufälligen Fahrzeugs aus der Liste
        pick = np.random.randint(0, len(choices))
        model = choices.index[pick]
        return model
    
    # drive() und charge() testen
    
    def drive(self, distance):
        trip_consumption = distance * (self.CONSUMPTION / 100) 
        self.SOC = self.SOC - (trip_consumption / self.CAPACITY) * 100
        
    def charge(self):
        # Fahrzeug befindet sich im Zielzustand des letzten Trips
        trip = self.trips[len(trips-1)]
        # nötige Zeit zum vollständigen Aufladen
        t_charge_full = ((100 - ev.SOC)/100 * ev.CAPACITY) / PCHARGE_SLOW
        # Ladezeit beschränkt durch Zeit zum vollständigen Aufladen und Länge des Aufenthalts
        t_charge = min(t_charge_full, trip.stay_duration)
        # update SOC über Ladedauer und Ladeleistung
        self.SOC = self.SOC + ((t_charge/60 * PCHARGE_SLOW) / self.CAPACITY) * 100
        trip.charge_start = trip.arrival
        trip.charge_end = trip.charge_start + t_charge

In [165]:
class Trip(object):
    
    def __init__(self, whyfrom):
        self.whyfrom = whyfrom
        self.whyto = None
        self.departure = None
        self.departure_t
        self.trip_duration = None
        self.arrival = None
        self.distance = None
        self.stay_duration = None
        self.charge_start = None 
        self.charge_end = None

In [166]:
# erzeugen Testfahrzeug, normalerweise Loop range(EV_GES)
ev = Electric_Vehicle()
# erzeugen Trip
ev_trip = Trip(whyfrom=1)

In [167]:
if TYPE_D == 1: 
    ev_trip.departure = int(np.round(initial_departure_model.sample()))
else:
    # Modell Samstag / Sonntag ergänzen
    pass
# Werte runden?

# diskreter Zeitschritt des Übergangs
ev_trip.departure_t = round(ev_trip.departure/T_STEP)

In [168]:
# wähle nächsten Zustand in Abhängigkeit des aktuellen Zustands und des Abfahrtszeitpunkt des Trips
if ev_trip.whyfrom == 1:
    ev_trip.whyto = np.random.choice([2, 3, 4, 5], p=transition_prob_home[ev_trip.departure_t])
    

In [169]:
# Samplen der Aufenthaltsdauer
ev_trip.stay_duration = stay_duration_model[ev_trip.whyto - 1].sample()

In [170]:
# Ermitteln der Wegstrecke
if TYPE_D == 1: 
    # Erläutering Datenstruktur siehe "Parameter der Verteilungsfunktion Wegstrecken:" in "Laden der Simulationsdaten"
    dist_shape = dist_wd_shape_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]
    dist_scale = dist_wd_scale_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]
    dist_loc = dist_wd_loc_ij[ev_trip.whyfrom - 1][ev_trip.whyto - 1]
    
    # samplen der lognorm Verteilungsfunktion der Zustandskombination
    ev_trip.distance = stats.lognorm.rvs(s=dist_shape, loc=dist_loc, scale=dist_scale)
else :
    # Werte Samstag / Sonntag 
    pass

In [171]:
# Placeholder Ladevorgang
if CHARGE_SCEN == 1:
    if ev.SOC < 100:
        # geladen wird nur wenn sich das Fahrzeug sich zuhause befindet (Szenario 1) und sich dort länger als 15 Minuten aufhält
        if ev.trip_no > 0 and ev.trips[len(ev.trips)-1].whyto == 1 and ev_trip.stay_duration > 15:
            ev.charge()
        
            
        

In [172]:
# Fahrvorgang -> update SOC 
ev.drive(ev_trip.distance)
ev_trip.trip_duration = (ev_trip.distance / EV_SPEED) * 60
ev_trip.arrival = ev_trip.departure + ev_trip.trip_duration
ev.trips.append(ev_trip)
ev.trip_no += 1

In [173]:
ev_trip.distance

7.732479058603191

In [174]:
ev.CAPACITY

30.0

In [175]:
ev.CONSUMPTION

14.3

In [176]:
ev.SOC

96.31418498206581